In [1]:
import pandas as pd
import numpy as np
import scipy
import time
import sys
sys.path.append('./gcn/gcn/')
from utils import *
import random
from models import *

In [2]:
# Set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)

# Settings
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 16, 'Number of units in hidden layer 1.')
flags.DEFINE_float('dropout', 0.5, 'Dropout rate (1 - keep probability).')
flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_integer('early_stopping', 10, 'Tolerance for early stopping (# of epochs).')
flags.DEFINE_integer('max_degree', 3, 'Maximum Chebyshev polynomial degree.')

In [8]:
W = pd.read_pickle('adjacency_small.pkl') 
H = pd.read_hdf('history_small.hdf', key='hist') 

In [10]:
W.shape, H.shape

((878, 878), (878, 290))

In [7]:
y_train_ = y_train[y_train.index.isin(idx)][idx_users]
#features_ = features[features.index.isin(idx)]
adj_ = adj[adj.index.isin(idx)][idx]
train_mask_ = np.array(list(map(bool, np.ones(len(adj_)))))

In [8]:
#with open('y_train.pkl', 'wb') as f:
#    pickle.dump(scipy.sparse.csr_matrix(y_train), f)

In [9]:
#y_train = hist + 1
y_train_ = y_train_.values
adj_[adj_<=0.3] = 0
adj_ = scipy.sparse.csr_matrix(adj_.values)
#features_ = scipy.sparse.lil.lil_matrix(features_.values)

In [10]:
y_train_.shape, adj_.shape, train_mask_.shape #features_.shape, 

((1000, 500), (1000, 1000), (1000,))

In [11]:
#features_ = preprocess_features(features_)

In [12]:
support_ = [preprocess_adj(adj_)]
num_supports = 1
model_func = GCN

In [13]:
# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    #'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features_[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, y_train_.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    'num_features_nonzero': tf.placeholder(tf.int32)  # helper variable for sparse dropout
}

In [16]:
# Create model
model = model_func(placeholders, input_dim=y_train_[2][1], logging=True)

# Initialize session
sess = tf.Session()

KeyError: 'features'

In [17]:
# Define model evaluation function
def evaluate(features, support, labels, mask, placeholders):
    t_test = time.time()
    feed_dict_val = construct_feed_dict(features, support, labels, mask, placeholders)
    outs_val = sess.run([model.loss, model.accuracy], feed_dict=feed_dict_val)
    return outs_val[0], outs_val[1], (time.time() - t_test)

In [16]:
# Init variables
sess.run(tf.global_variables_initializer())

cost_val = []

# Train model
for epoch in range(FLAGS.epochs):

    t = time.time()
    # Construct feed dictionary
    feed_dict = construct_feed_dict(features_, support_, y_train_, train_mask_, placeholders)
    feed_dict.update({placeholders['dropout']: FLAGS.dropout})

    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy], feed_dict=feed_dict)

    # Validation
    cost, acc, duration = evaluate(features_, support_, y_train_, train_mask_, placeholders)
    cost_val.append(cost)

    # Print results
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > FLAGS.early_stopping and cost_val[-1] > np.mean(cost_val[-(FLAGS.early_stopping+1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished!")

Epoch: 0001 train_loss= 59.15310 train_acc= 0.00000 val_loss= 59.08974 val_acc= 0.00000 time= 8.22234
Epoch: 0002 train_loss= 59.09385 train_acc= 0.00000 val_loss= 59.02153 val_acc= 0.00000 time= 2.25967
Epoch: 0003 train_loss= 59.02235 train_acc= 0.00000 val_loss= 58.94949 val_acc= 0.00000 time= 3.25773
Epoch: 0004 train_loss= 58.95266 train_acc= 0.00000 val_loss= 58.87394 val_acc= 0.00000 time= 2.38136
Epoch: 0005 train_loss= 58.87285 train_acc= 0.00000 val_loss= 58.79516 val_acc= 0.00000 time= 2.31219
Epoch: 0006 train_loss= 58.79160 train_acc= 0.00000 val_loss= 58.71335 val_acc= 0.00000 time= 2.29659
Epoch: 0007 train_loss= 58.71661 train_acc= 0.00000 val_loss= 58.62879 val_acc= 0.00000 time= 4.19655
Epoch: 0008 train_loss= 58.67253 train_acc= 0.00000 val_loss= 58.54195 val_acc= 0.00000 time= 2.40630
Epoch: 0009 train_loss= 58.52223 train_acc= 0.00000 val_loss= 58.45323 val_acc= 0.00000 time= 2.60343
Epoch: 0010 train_loss= 58.48873 train_acc= 0.00000 val_loss= 58.36312 val_acc= 0.

In [1]:
(59.15310 - 55.96139 )/59.15310

0.05395676642475205